In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import cross_val_score,StratifiedKFold,KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
import joblib

In [2]:
df = pd.read_csv('../data/train.csv')
df

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [3]:
# 数据集划分
X_columns = df.columns.drop(['id', 'target'])
y_column = ['target']

In [4]:
# 损失函数
#def micro_f1_scorer(y, y_pred):
#    return f1_score(y, y_pred, average='micro')

def gini(actual, pred):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float64)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

gini_normalized_score = make_scorer(gini_normalized)

In [5]:
# 模型训练
model = LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=200)

In [6]:
X = df[X_columns]
y = df[y_column]

In [8]:
# 训练模型
model.fit(df[X_columns], df[y_column])

LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=200)

In [13]:
feature_importance = pd.DataFrame({'feature': X_columns, 'feature_importances': model.feature_importances_})
feature_importance

,feature,feature_importances
0,ps_ind_01,234
1,ps_ind_02_cat,125
2,ps_ind_03,390
3,ps_ind_04_cat,103
4,ps_ind_05_cat,298
5,ps_ind_06_bin,18
6,ps_ind_07_bin,44
7,ps_ind_08_bin,40
8,ps_ind_09_bin,20
9,ps_ind_10_bin,2


In [14]:
feature_importance.sort_values('feature_importances', ascending=False)

,feature,feature_importances
20,ps_reg_03,398
2,ps_ind_03,390
34,ps_car_13,380
4,ps_ind_05_cat,298
0,ps_ind_01,234
14,ps_ind_15,232
35,ps_car_14,231
33,ps_car_12,191
19,ps_reg_02,187
21,ps_car_01_cat,167


In [20]:
selected_features = [feature for feature in X_columns if feature_importance[feature_importance['feature']==feature]['feature_importances'].values[0]>=20]

In [22]:
print(len(X_columns))
print(len(selected_features))

57
45


In [24]:
# 模型训练
model = LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=200)

In [26]:
model.fit(df[selected_features], df[y_column])

LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=200)

In [28]:
pred_data = pd.read_csv('../data/test.csv')
submit_data = pd.DataFrame()
submit_data['id'] = pred_data['id']
submit_data['target'] = model.predict(pred_data[selected_features])

submit_data.to_csv('lgb_best_param_select_feature_submission.csv', index=False, sep=',')

In [29]:
selected_features

['ps_ind_01',
 'ps_ind_02_cat',
 'ps_ind_03',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_12_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_11_cat',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'ps_calc_01',
 'ps_calc_02',
 'ps_calc_03',
 'ps_calc_04',
 'ps_calc_05',
 'ps_calc_06',
 'ps_calc_07',
 'ps_calc_08',
 'ps_calc_09',
 'ps_calc_10',
 'ps_calc_11',
 'ps_calc_12',
 'ps_calc_13',
 'ps_calc_14']